In [60]:
import os

In [61]:
import hail as hl

localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

hl.init(
    tmp_dir=localfs_path,
    local_tmpdir=localfs_path
       )

2023-10-26 15:00:08.088 Hail: WARN: Hail has already been initialized. If this call was intended to change configuration, close the session with hl.stop() first.


FatalError: IllegalArgumentException: requirement failed

Java stack trace:
java.lang.IllegalArgumentException: requirement failed
	at scala.Predef$.require(Predef.scala:268)
	at is.hail.backend.spark.SparkBackend$.apply(SparkBackend.scala:226)
	at is.hail.backend.spark.SparkBackend.apply(SparkBackend.scala)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnecti

In [92]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis
from scipy import stats
import seaborn as sns

output_notebook()

Loading BokehJS ...

Loading BokehJS ...

In [ ]:
import sys
!{sys.executable} -m pip install seaborn

In [ ]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-all.mt')

In [ ]:
sport_pheno = hl.import_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/sportsmen-pheno.csv', impute = True, key='sample_id', quote ="\"")
mt = mt.annotate_cols(sport_phenotypes = sport_pheno[mt.s])

In [9]:
mt = hl.variant_qc(mt)
mt.checkpoint(localfs_path+'qc2.mt')

2023-10-24 14:05:30.470 Hail: INFO: wrote matrix table with 6514756 rows and 280 columns in 488 partitions to /localfs/5684153/qc2.mt


In [10]:
mt = mt.filter_rows(hl.is_snp(
    mt.alleles[0], mt.alleles[1])
                   )
mt.checkpoint(localfs_path+'snps2.mt')

2023-10-24 14:07:35.159 Hail: INFO: wrote matrix table with 6514756 rows and 280 columns in 488 partitions to /localfs/5684153/snps2.mt


In [11]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [12]:
mt = mt.checkpoint(localfs_path+'all_snps2.mt')

2023-10-24 14:09:40.468 Hail: INFO: wrote matrix table with 6514756 rows and 280 columns in 488 partitions to /localfs/5684153/all_snps2.mt


In [13]:
mt.count()

(6514756, 280)

In [14]:
mt.checkpoint('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/fanci/all-snps.mt')

2023-10-24 14:18:23.639 Hail: INFO: wrote matrix table with 6514756 rows and 280 columns in 488 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/fanci/all-snps.mt


In [16]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/fanci/all-snps.mt')

### variants near our variant of interest:

In [17]:
near = mt.filter_rows(
    (mt.locus.contig == "chr15") &
    (mt.locus.position > (89315318-100000)) &
    (mt.locus.position < (89315318+100000))
) # +/- 100 kb

In [18]:
near.checkpoint('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/fanci/vars-near-fanci.mt')

2023-10-24 14:34:11.159 Hail: INFO: wrote matrix table with 685 rows and 280 columns in 488 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/fanci/vars-near-fanci.mt


### "our fanci variant"

In [24]:
voi = near.filter_rows(
    (near.locus.contig == "chr15") &
    (near.locus.position == 89315318)
)

In [26]:
voi = voi.distinct_by_row()

In [27]:
voi.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/fanci/fanci-var.mt', overwrite = True)

2023-10-24 15:22:44.434 Hail: INFO: wrote matrix table with 1 row and 280 columns in 488 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/fanci/fanci-var.mt


In [28]:
voi.show()

,
locus,alleles
locus<GRCh38>,array<str>
chr15:89315318,"[""C"",""T""]"


In [31]:
voi.count()

(1, 280)

In [29]:
voi.variant_qc.show()

+----------------+------------+--------------------------+
| locus          | alleles    | variant_qc.dp_stats.mean |
+----------------+------------+--------------------------+
| locus<GRCh38>  | array<str> |                  float64 |
+----------------+------------+--------------------------+
| chr15:89315318 | ["C","T"]  |                 2.94e+01 |
+----------------+------------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.dp_stats.stdev | variant_qc.dp_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  1.02e+01 |                2.00e+00 |
+---------------------------+-------------------------+

+-------------------------+--------------------------+
| variant_qc.dp_stats.max | variant_qc.gq_stats.mean |
+-------------------------+--------------------------+
|                 float64 |                  float64 |
+-------------------------+--------------------------+
|                5.80e+01 |                 7.49e+01 |
+-------------------------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.gq_stats.stdev | variant_qc.gq_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  2.24e+01 |                0.00e+00 |
+---------------------------+-------------------------+

+-------------------------+---------------+---------------------+
| variant_qc.gq_stats.max | variant_qc.AC | variant_qc.AF       |
+-------------------------+---------------+---------------------+
|                 float64 | array<int32>  | array<float64>      |
+-------------------------+---------------+---------------------+
|                9.90e+01 | [557,3]       | [9.95e-01,5.36e-03] |
+-------------------------+---------------+---------------------+

+---------------+-----------------------------+----------------------+
| variant_qc.AN | variant_qc.homozygote_count | variant_qc.call_rate |
+---------------+-----------------------------+----------------------+
|         int32 | array<int32>                |              float64 |
+---------------+-----------------------------+----------------------+
|           560 | [277,0]                     |             1.00e+00 |
+---------------+-----------------------------+----------------------+

+---------------------+-------------------------+-----------------------+
| variant_qc.n_called | variant_qc.n_not_called | variant_qc.n_filtered |
+---------------------+-------------------------+-----------------------+
|               int64 |                   int64 |                 int64 |
+---------------------+-------------------------+-----------------------+
|                 280 |                       0 |                     0 |
+---------------------+-------------------------+-----------------------+

+------------------+----------------------+-------------------------+
| variant_qc.n_het | variant_qc.n_non_ref | variant_qc.het_freq_hwe |
+------------------+----------------------+-------------------------+
|            int64 |                int64 |                 float64 |
+------------------+----------------------+-------------------------+
|                3 |                    3 |                1.07e-02 |
+------------------+----------------------+-------------------------+

+------------------------+-------------------------------+
| variant_qc.p_value_hwe | variant_qc.p_value_excess_het |
+------------------------+-------------------------------+
|                float64 |                       float64 |
+------------------------+-------------------------------+
|               5.03e-01 |                      4.97e-01 |
+------------------------+-------------------------------+

In [ ]:
p = hl.plot.histogram(voi.DP, range=(0, 65), bins=30)

In [ ]:
show(p)

In [48]:
voi = voi.annotate_cols(
    group = hl.if_else(
        voi.s.contains('B'), 'sport', 'GTS'
    )
)

In [ ]:
voi_sport = voi.filter_cols(voi.group == 'sport')

In [50]:
ps = hl.plot.histogram(voi_sport.DP, range=(0, 65), bins=30)

In [51]:
show(ps)

In [57]:
p2 = hl.plot.histogram(voi_sport.GQ, range=(0, 100), bins=30)

In [58]:
show(p2)

In [67]:
voi = voi.annotate_cols(
    het = hl.agg.any(voi.GT.is_non_ref())
)

In [68]:
voi_non_ref = voi.filter_cols(voi.het)

In [69]:
voi_non_ref.s.collect()

['B439', 'B480', 'B482']

In [14]:
near = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/fanci/vars-near-fanci.mt')

### Are there any variants nearby also shared by these three samples?

In [81]:
poles = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/resources/polish-genomes/polish-genomes.mt')

In [82]:
near = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/fanci/vars-near-fanci.mt')

In [83]:
near = near.annotate_rows(polish_af = poles.rows()[near.row_key]['info'])

In [84]:
near = near.distinct_by_row()

In [85]:
my_samples = near.filter_cols(hl.literal(['B439', 'B480', 'B482']).contains(near.s))

In [86]:
my_samples = my_samples.filter_rows(
    hl.agg.all(my_samples.GT.is_hom_ref()) |
    hl.agg.all(my_samples.GT.is_het()) |
    hl.agg.all(my_samples.GT.is_hom_var())
)

In [87]:
near = near.filter_rows(hl.is_defined(my_samples.rows()[near.row_key]))

In [23]:
near.checkpoint(localfs_path+'haplo.mt')

2023-10-25 12:39:44.180 Hail: INFO: wrote matrix table with 412 rows and 280 columns in 488 partitions to /localfs/5702757/haplo.mt


In [88]:
near = hl.read_matrix_table(localfs_path+'haplo.mt')

In [89]:
near = near.distinct_by_row()

In [90]:
near = near.annotate_rows(het_non_refs = hl.agg.group_by(near.group, hl.agg.count_where(near.GT.is_het())))
near = near.annotate_rows(hom_refs = hl.agg.group_by(near.group, hl.agg.count_where(near.GT.is_hom_ref())))
near = near.annotate_rows(hom_non_refs = hl.agg.group_by(near.group, hl.agg.count_where(near.GT.is_hom_var())))

In [91]:
near = near.annotate_rows(het_non_refs_e_vs_s = hl.agg.group_by(near.sport_phenotypes.type, hl.agg.count_where(near.GT.is_het())))
near = near.annotate_rows(hom_refs_e_vs_s = hl.agg.group_by(near.sport_phenotypes.type, hl.agg.count_where(near.GT.is_hom_ref())))
near = near.annotate_rows(hom_non_refs_e_vs_s = hl.agg.group_by(near.sport_phenotypes.type, hl.agg.count_where(near.GT.is_hom_var())))

In [92]:
sport_AC_non_ref = hl.int32(near.het_non_refs.get('sport', 0) + (near.hom_non_refs.get('sport', 0)*2))
sport_AC_ref = hl.int32(near.het_non_refs.get('sport', 0) + (near.hom_refs.get('sport', 0)*2))
control_AC_non_ref = hl.int32(near.het_non_refs.get('GTS', 0) + (near.hom_non_refs.get('GTS', 0)*2))
control_AC_ref = hl.int32(near.het_non_refs.get('GTS', 0) + (near.hom_refs.get('GTS', 0)*2))
speed_AC_non_ref = hl.int32(near.het_non_refs_e_vs_s.get('speed', 0) + (near.hom_non_refs_e_vs_s.get('speed', 0)*2))
speed_AC_ref = hl.int32(near.het_non_refs_e_vs_s.get('speed', 0) + (near.hom_refs_e_vs_s.get('speed', 0)*2))
endurance_AC_non_ref = hl.int32(near.het_non_refs_e_vs_s.get('endurance', 0) + (near.hom_non_refs_e_vs_s.get('endurance', 0)*2))
endurance_AC_ref = hl.int32(near.het_non_refs_e_vs_s.get('endurance', 0) + (near.hom_refs_e_vs_s.get('endurance', 0)*2))
gnomad_AC_non_ref = near.gnomad_v_3_1.freq.AC[2]
gnomad_AC_ref = near.gnomad_v_3_1.freq.AN[2] - near.gnomad_v_3_1.freq.AC[2]
polish_AC_non_ref = hl.int32(near.polish_af.AC[0])
polish_AC_ref = hl.int32(near.polish_af.AN - near.polish_af.AC[0])

near = near.annotate_rows(fisher_sport_vs_gts = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, control_AC_non_ref, control_AC_ref),
                      fisher_e_vs_s = hl.fisher_exact_test(speed_AC_non_ref, speed_AC_ref, endurance_AC_non_ref, endurance_AC_ref),
                      fisher_sport_vs_gnomad = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, gnomad_AC_non_ref, gnomad_AC_ref),
                      fisher_sport_vs_polish = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, polish_AC_non_ref, polish_AC_ref))

near = near.drop(near['a_index'], near['was_split'])

In [93]:
near = near.distinct_by_row()

In [94]:
near = near.distinct_by_col()

## What is the ld of "our variant" with other variants:

In [95]:
ld = hl.ld_matrix(near.GT.n_alt_alleles(), near.locus, radius=2e6)

2023-10-25 20:37:03.330 Hail: INFO: Wrote all 1 blocks of 412 x 279 matrix with block size 4096.


In [100]:
ld_near = ld.to_numpy()

In [104]:
ld_near.shape

(412, 412)

In [107]:
near = near.add_row_index()

In [116]:
near.row_idx.take(5)

[0, 1, 2, 3, 4]

In [113]:
near.filter_rows(
    (near.locus.contig == "chr15") &
    (near.locus.position == 89315318)
).row_idx.take(5) # this is the index of my variant

[235]

In [119]:
pd.DataFrame(ld_near[235]).to_csv('ld.csv')

In [65]:
near = near.transmute_rows(DP_stats = near.variant_qc.dp_stats,
                           GQ_stats = near.variant_qc.gq_stats,
                           hwe_p_value = near.variant_qc.p_value_hwe,
                           gnomad_v3_nfe_af = near.gnomad_v_3_1.freq.AF[2],
                           gnomad_v3_nfe_homozygote_count = near.gnomad_v_3_1.freq.homozygote_count[2],
                           cadd = near.cadd.score_phred,
                           within_gene = near.vep.vep.transcript_consequences.gene_symbol,
                           fisher_sport_vs_gts  = near.fisher_sport_vs_gts,
                           fisher_e_vs_s = near.fisher_e_vs_s,
                           fisher_sport_vs_gnomad  = near.fisher_sport_vs_gnomad,
                           fisher_sport_vs_polish = near.fisher_sport_vs_polish,
                           het_sport = near.het_non_refs.get('sport', 0),
                           het_gts = near.het_non_refs.get('GTS', 0),
                           hom_ref_sport = near.hom_refs.get('sport', 0),
                           hom_ref_controls = near.hom_refs.get('GTS', 0),
                           hom_var_sport = near.hom_non_refs.get('sport', 0),
                           hom_var_gts = near.hom_non_refs.get('GTS',0),
                           het_endurance = near.het_non_refs_e_vs_s.get('endurance', 0),
                           het_speed = near.het_non_refs_e_vs_s.get('speed', 0),
                           hom_ref_endurance = near.hom_refs_e_vs_s.get('endurance', 0),
                           hom_ref_speed = near.hom_refs_e_vs_s.get('speed', 0),
                           hom_var_endurance = near.hom_non_refs_e_vs_s.get('endurance', 0),
                           hom_var_speed = near.hom_non_refs_e_vs_s.get('speed',0),
                           most_severe_consequence = near.vep.vep.most_severe_consequence,
                           transcript_consequences = near.vep.vep.transcript_consequences,
                           intergenic_consequences = near.vep.vep.intergenic_consequences,
                           motif_feature_consequences = near.vep.vep.motif_feature_consequences,
                           regulatory_feature_consequences = near.vep.vep.regulatory_feature_consequences,
                           polish_af = near.polish_af)

near = near.select_entries(near.GT)

near = near.distinct_by_col()
near = near.make_table()
near = near.checkpoint(localfs_path+'to_export_near.mt', overwrite = True)

2023-10-25 14:38:20.926 Hail: INFO: wrote table with 412 rows in 488 partitions to /localfs/5702757/to_export_near.mt


In [66]:
to_export = near.flatten()
to_export.export('haplo.csv')

2023-10-25 15:17:33.678 Hail: INFO: merging 489 files totalling 4.8M...1) / 488]
2023-10-25 15:17:33.759 Hail: INFO: while writing:
    haplo.csv
  merge time: 80.295ms


# Kinship analysis for all of the samples:

In [7]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/fanci/all-snps.mt')
mt = mt.naive_coalesce(100)

In [8]:
mt = mt.sample_rows(0.1)

In [9]:
mt = mt.checkpoint(localfs_path+'subset_rand')

2023-10-26 09:41:39.433 Hail: INFO: wrote matrix table with 650506 rows and 280 columns in 100 partitions to /localfs/5715784/subset_rand
    Total size: 4.56 GiB
    * Rows/entries: 4.56 GiB
    * Columns: 35.28 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  12449 rows (94.14 MiB)


In [ ]:
pc_rel2 = hl.pc_relate(mt.GT, 0, k=10, statistics='kin')
pc_rel2 = pc_rel2.checkpoint(localfs_path+'pc_rel2.mt')

In [18]:
pc_rel3 = hl.pc_relate(mt.GT, 0.05, k=10, statistics='kin')
pc_rel3 = pc_rel3.checkpoint(localfs_path+'pc_rel3.mt')

2023-10-26 10:01:12.906 Hail: INFO: hwe_normalize: found 645890 variants after filtering out monomorphic sites.
2023-10-26 10:01:22.577 Hail: INFO: pca: running PCA with 10 components.../ 100]
2023-10-26 10:05:02.150 Hail: INFO: wrote table with 0 rows in 0 partitions to /localfs/5715784/persist_tabledVJ18v6mXt
    Total size: 23.50 KiB
    * Rows: 0.00 B
    * Globals: 23.50 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A
2023-10-26 10:05:18.304 Hail: INFO: Wrote all 159 blocks of 650506 x 280 matrix with block size 4096.
2023-10-26 10:05:20.150 Hail: INFO: wrote matrix with 11 rows and 650506 columns as 159 blocks of size 4096 to /localfs/5715784//pcrelate-write-read-X3T0igHpBzfhkTRJT1GDfC.bm
2023-10-26 10:05:21.294 Hail: INFO: wrote matrix with 650506 rows and 280 columns as 159 blocks of size 4096 to /localfs/5715784//pcrelate-write-read-PsyVe8xVNSqVeBdt7T3f5h.bm
2023-10-26 10:05:45.420 Hail: INFO: wrote matrix with 280 rows and 280 columns as 1 block of size 4096 t

In [19]:
pc_rel4 = hl.pc_relate(mt.GT, 0.01, k=10, statistics='kin')
pc_rel4 = pc_rel4.checkpoint(localfs_path+'pc_rel4.mt')

2023-10-26 10:06:19.043 Hail: INFO: hwe_normalize: found 645890 variants after filtering out monomorphic sites.
2023-10-26 10:06:24.641 Hail: INFO: pca: running PCA with 10 components.../ 100]
2023-10-26 10:09:51.297 Hail: INFO: wrote table with 0 rows in 0 partitions to /localfs/5715784/persist_tableY37Ob5xeme
    Total size: 23.51 KiB
    * Rows: 0.00 B
    * Globals: 23.51 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A
2023-10-26 10:10:08.165 Hail: INFO: Wrote all 159 blocks of 650506 x 280 matrix with block size 4096.
2023-10-26 10:10:10.023 Hail: INFO: wrote matrix with 11 rows and 650506 columns as 159 blocks of size 4096 to /localfs/5715784//pcrelate-write-read-PyGuoRUlR7Kd3lvEfoon8X.bm
2023-10-26 10:10:11.324 Hail: INFO: wrote matrix with 650506 rows and 280 columns as 159 blocks of size 4096 to /localfs/5715784//pcrelate-write-read-X5c5cNKdAU6t2rYzwdGjlL.bm
2023-10-26 10:10:41.576 Hail: INFO: wrote matrix with 280 rows and 280 columns as 1 block of size 4096 t

In [21]:
our_s = ['B439', 'B480', 'B482']

In [49]:
our_filt = (hl.literal(our_s).contains(pc_rel4.i.s)) & (hl.literal(our_s).contains(pc_rel4.j.s))

In [62]:
pc_rel4.filter(our_filt).show()

,,
i,j,
s,s,kin
str,str,float64
"""B439""","""B480""",-2.82e-03
"""B439""","""B482""",-4.50e-04
"""B480""","""B482""",-2.76e-03


### How many SNPs in 2, 3 and 4 sportmen

In [79]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/fanci/all-snps.mt')

In [80]:
mt = mt.annotate_cols(
    group = hl.if_else(
        mt.s.contains('B'), 'sport', 'GTS'
    )
)

In [81]:
sport = mt.filter_cols(mt.group == 'sport') 

In [82]:
sport = sport.filter_rows(hl.agg.any(sport.GT.is_non_ref()))

In [84]:
sport = sport.checkpoint(localfs_path+'sport-only.mt', overwrite = True)

2023-10-26 16:41:49.747 Hail: INFO: wrote matrix table with 4452374 rows and 100 columns in 488 partitions to /localfs/5715784/sport-only.mt


In [85]:
sport = sport.annotate_rows(
    var_sum = hl.agg.sum(sport.GT.is_non_ref())
)

In [86]:
sport2 = sport.filter_rows(sport.var_sum == 2)

In [87]:
sport2.count()

(319689, 100)

In [88]:
sport3 = sport.filter_rows(sport.var_sum == 3)

In [89]:
sport3.count()

(180783, 100)

In [103]:
s4 = hl.plot.histogram(sport4.rows().gnomad_v_3_1.freq.AF[2], range=(0, 0.2), bins=30)

In [104]:
show(s4)

In [90]:
sport4 = sport.filter_rows(sport.var_sum == 4)

In [91]:
sport4.count()

(131612, 100)